# CSS Fuzzing Results Analysis

This notebook loads CSS fuzzer JSON results from multiple devices and browsers, aligns expressions across devices, and identifies the top expressions by their discriminating power (variance) and by entropy.

In [1]:
import hashlib
import os
import glob
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [2]:
# Set this to the folder containing your JSON result files
data_dir = 'src_files'
json_files = glob.glob(os.path.join(data_dir, '*.json'))
print(f"Found {len(json_files)} JSON files.")


Found 49 JSON files.


In [3]:
records = []
for filepath in json_files:
    with open(filepath, 'r') as f:
        data = json.load(f)
    meta = data.get('meta', {})
    # create a hash from the entire meta dict
    meta_str = json.dumps(meta, sort_keys=True)
    device_id = f"{meta['host']}_{meta['os']}_{meta['browser']}_{meta['release']}"
    for rec in data.get('results', []):
        expr = rec.get('expression')
        res_str = rec.get('result', '')
        # Remove 'px' and convert to float
        try:
            res_val = float(res_str.strip().removesuffix('px'))
        except:
            try:
                res_val = float(res_str.strip().replace('px',''))
            except:
                res_val = np.nan
        records.append({'expression': expr, 'device': device_id, 'result': res_val})

# Create DataFrame
rec_df = pd.DataFrame(records)
rec_df.head()


,expression,device,result
0,( 0.31358980131335557 / pi - ( 32325.63012...,iphone13_ios_brave_1.61.1,0.000000
1,37414.684795947 / ( pi * sin( 0.2556375565...,iphone13_ios_brave_1.61.1,38124.265625
2,0.2889663081150502 / e * e,iphone13_ios_brave_1.61.1,0.281250
3,( 0.7106291791424155 * 0.7406447387766093 ) ...,iphone13_ios_brave_1.61.1,0.000000
4,104984.34237470584 / e + e * ( e / pi ...,iphone13_ios_brave_1.61.1,0.000000


In [4]:
# ——————————————————————————————————————————————————————————————
# Find duplicate (expression, device) pairs with inconsistent results

# 1) Compute how many distinct results each (expr,device) has
counts = (
    rec_df
    .groupby(['expression', 'device'])['result']
    .nunique()
    .reset_index(name='distinct_result_count')
)

# 2) Keep only those with more than one distinct result
bad_pairs = counts[counts['distinct_result_count'] > 1]

# 3) Extract all raw rows belonging to those bad pairs
dup_inconsistencies = rec_df.merge(
    bad_pairs[['expression', 'device']],
    on=['expression', 'device'],
    how='inner'
).sort_values(['expression', 'device'])

# Show them and raise an error
if not dup_inconsistencies.empty:
    display(dup_inconsistencies)
    raise ValueError("Inconsistent results found for some (expression, device) pairs.")
else:
    print("All duplicate (expression,device) entries agree on result.")

All duplicate (expression,device) entries agree on result.


## Pivot DataFrame to have expressions as rows and devices as columns

In [5]:
pivot_df = rec_df.pivot_table(
    index='expression',
    columns='device',
    values='result',
    aggfunc='mean'    # or 'first', 'min', 'max'…
)


## Compute Variance Across Devices

In [6]:
variance_series = pivot_df.var(axis=1, skipna=True)
variance_df = variance_series.to_frame('variance').sort_values('variance', ascending=False)
variance_df.head(20)


,variance
expression,
32970.133250076935 * ( e * 21331.302134109705 + sin( pi * 2426.5901943005865 / ( 113385.35342747977 ) - 54949.65054734511 / 0.572849080665037 * -59627.8480884388 + tan( -11797880.755120402 ) / cos( 32991.56475314037 ) ) ) / tan( 91858.34696446847 / cos( 259169.5286292584 - sin( 83803.39611051466 ) - 2282.303712737041 ) ) + pi * 0.3770266817882657,2.244013e+14
144116.27305271316 * pi / e + pi * e * sin( ( tan( 73424.786949128 ) * 0.9439510465599597 / ( 56530.61477244103 ) - pi ) * 255464.21610215 / 7119.375702326182 - 0.5130449610296637 * tan( pi / tan( 55629.497388729535 ) - e * e ) + pi / 80600.1643548241 ) + ( tan( ( 344097.42149705905 ) ) * cos( 66626.91447564634 * 142554.71793764457 + 408804.31088346115 ) * 217339.59344884998 + ( 73348.94143565705 * 19500.567223032776 / 84981.18549244491 + 284612.24463356653 * -25003.225251357995 ) / 827187.116865794 - infinity / e * ( sin( 912573.6511624103 ) + 119707.6811974998 / 0.9168360356707126 ) ) * 130212.05105774822 / ( cos( 2550.9780349103166 / 0.11417809268459678 * tan( 170976.83206274835 ) + 0.3416550566907972 / 129460.99766829397 ) / 46250.55158144567 / 65848.97454139592 ),2.244013e+14
e / 51792.926274419005 / 292754.9351654965 + ( cos( ( 50192.10938532508 ) * 343381.37186629564 / 0.9428970818407834 ) * 0.8273021187633276 / ( 0.12080961861647666 * 0.521072308300063 - 0.29102701018564403 ) ) - tan( sin( 0.2602045340463519 / tan( 66601.06102373435 ) ) * ( 69830.08841963652 * sin( 334295.37086600193 ) ) / pi ) / 117350.57132645075 / ( 0.0020127128809690475 / ( 18032.3337984323 - 0.07703996892087162 / 0.22185992076992989 + 1349727.662409458 * 84720.58775373698 ) ),2.244013e+14
e / 0.8083986975252628 / ( 429444.77165502403 / sin( 98808.98904537983 / cos( 239493.99768523243 ) ) + 0.9500930760987103 / ( 0.19637086032889783 * 0.5169258562382311 * ( 205062.81970092517 ) ) / cos( 37147.44541188763 / ( 841430.1787131258 ) + 0.14586688857525587 * sin( 172499.07805386206 ) ) ) - ( ( cos( 69477.1999171126 ) / e - 0.014860054245218635 / ( 17887.92435889667 ) + 300743.00208643294 * pi * ( 43299.21589925185 ) ) - tan( 1816.1981264709827 / 0.8418901602271944 + 0.5105363873299211 / 175281.54605080388 / 187607.8017576784 ) + infinity / 181885.94153351744 / sin( cos( 131975.91989331678 ) / 56622.784323933505 - ( -54340801.27615317 ) / 104452.04847617635 + 78795.48426028604 * 0.9188502880278975 * ( 24707.355183412452 ) ) ) - 63207.87831575034 * ( 4723.83455082577 / 79856.61634919904 - pi ),2.244013e+14
156780.27200292773 / e * ( 157766.20962136713 + 140960.9830923717 / tan( -25348.602231715395 - ( 547894.3535073622 ) * 870.9039561331771 ) / 0.7385085606947541 ) - ( 40193.629828694735 * 101551.90495629811 * 0.9660667572170496 - 0.9047497091814876 - pi * e ) / ( 0.6578597927000374 - 163248.54100633305 * sin( 199168.54673455376 * tan( 131798.73773057092 ) / cos( 53095.94215184761 ) ) ) / ( pi ),2.244013e+14
( e / sin( ( 7822.046031714993 ) - 247036.81504952986 * 0.383386337896809 ) - 11856.619339385543 * 65417.86335882549 / ( 0.840907184407115 * 222740.33950751647 + e + pi / ( 153634.2365913437 ) ) + ( 113533.46217192124 ) ) * ( ( ( 77509.65101392985 ) - 0.46244007046334445 * ( 138460.2399282894 ) - e * ( 2672.1510906169665 ) * infinity ) / tan( 65845.77103623672 * 5860.264102150098 * 324349.93660747743 ) - 0.4010413638316095 / ( ( 58118.376286106686 ) - e - 0.11916521098464727 / 0.08358802087605 ) ) / pi + tan( 114943.43563296756 ) * 101124.51455956457 * tan( 260577.13418472867 ),2.244013e+14
sin( 120511.62227241456 * 82109.04739416987 + 163933.5672223619 / ( ( 330467.1532101026 ) - 2589147.008702339 / pi ) * 0.20643631252460182 ) / cos( 570863.9401198282 / 12253.879804313407 - 0.4794786931015551 ) / 0.4600610239431262 + cos( 115425.85986848721 / tan( sin( 193521.25273909446 ) + 0.46031457907520235 * 38875.946571342865 * 107861.57008425212 - 73733.34565752839 / sin( 56909.151822983906 ) * 70774.87102534215 ) + 0.39262618822976947 * ( ( 287409.56034314155 ) * 71960.9762548649 * (

## Compute Entropy Across Devices

In [7]:
from math import log2

entropy_list = []
for expr, row in pivot_df.iterrows():
    vals = row.dropna().values
    if len(vals) == 0:
        ent = np.nan
    else:
        counts = pd.Series(vals).value_counts()
        probs = counts / counts.sum()
        ent = -(probs * np.log2(probs)).sum()
    entropy_list.append((expr, ent))

entropy_df = pd.DataFrame(entropy_list, columns=['expression','entropy']).set_index('expression')
entropy_df = entropy_df.sort_values('entropy', ascending=False)
entropy_df.head(20)


,entropy
expression,
0.836337408516556 / sin( ( 328660.6002077978 * pi + ( 169101.13197941292 ) ) * ( 121897.3210943361 * 59011.5150966451 * 33744.317029065365 + 101130.22947068278 / ( 247623.22206646253 ) / 0.8113027140498161 ) ) + e,2.868240
0.26773800421506166 - 47347.7351729909 / 69806.10740005101 - e * tan( pi + 54370.98221018599 / tan( 145.19201052953946 ) * 0.44596617901697755 * 135917.42503542083 ) / pi,2.841861
0.09127735486254096 / 735.8453242566477 - sin( 81929.39040861855 / 0.7371127642691135 ) * sin( pi - 0.0050916902255266905 ) * ( pi * 56598.82715263593 - tan( 138618.8207329427 - sin( 87866.92300892183 ) ) * 73963.6481315199 * sin( 38350.44718330214 * 88426.60347159175 * 70748.46378042221 + e * 0.3522350515704602 ) - 155284.45749961227 / 76410.38662212781 / tan( 0.6574289293494076 * cos( 5566943.480322141 ) - e / 27243.048928791104 ) ) + ( e ) * ( 0.767688465770334 - e / sin( 78576.17265717337 * 357461.36174224137 + 81647.98113721207 ) ),2.841861
pi * tan( -172362.5349671105 * cos( 106590.18812615762 ) - -135310.96249555165 / 0.43122277199290693 - ( sin( 35.65965735952698 ) + ( 39092.58851640222 ) / 143281.34438369097 ) * 25952.2169060161 * ( pi ) ),2.822718
0.7887798345182091 * sin( 82189.05545839571 / 0.18517374992370605 * 0.008089403854683042 ) / tan( tan( 109291.67664288744 * ( 36295.7350248317 ) - 204664.3786005878 - 209845.36376121573 * 0.7465927922166884 ) * 184123.336033595 ) + ( sin( ( 131293.05520239723 ) - 150286.96096293302 * 0.8082891888916492 ) / 0.777242538286373 + 0.5813473116140813 ) * 0.6146595270838588 * tan( 0.7512196360621601 / 8514.927909941202 + ( 136784.9486422678 ) / ( 0.5166313732042909 * 105161.13063391473 / 0.6193476456683129 ) + 0.599509299499914 * 467214.32023847103 ),2.813943
( 819089.5583285954 / 56608.20596055245 ) / cos( pi + sin( 76236.68535730825 / 40736.93352735403 ) ) / 130187.78693887388 * 649103.1720475996 / sin( ( 211870.47864979794 ) * 493475.8066673059 ),2.813943
2177121.019011976 / 134882.69539745056 / 0.9244347715284675 - ( cos( e * ( 191596.40883588613 ) + 262619.3231425485 * 274051.0595224638 ) / 0.328251866158098 ) * sin( ( 11588.896119517485 * 32314.82891819235 * cos( 20434908.45764997 ) ) * pi * 144196.89544439805 + 0.6858788279350847 * ( 110257.92776979548 ) ),2.813943
e + 3875368.0213615815 / sin( 0.04519110103137791 * 0.7010521683841944 - ( e * 2173041.0204142155 - 113148.70869192715 * 23930.135977992977 ) + 1280812.8273997805 * cos( 953810.3230102585 / e - ( 379086.80439042306 ) - 380405.12721971783 / sin( 137104.94121932628 ) ) ) / -77576.96395318274,2.813943
tan( e / ( 246224.70135592142 + ( 13850.590343604119 ) ) - cos( pi / -45285.84589270301 - 294211.75868571043 * 140089.91514788382 ) * -150374.04913732802 ) + sin( cos( pi / 29327.001922664764 - -1015194.7508332379 / -infinity ) ) * sin( 382429.31002737605 ) - tan( 0.0010620588436722755 * 0.8560699264053255 * 441999.98665663926 + 0.7504787261132151 / cos( 102703.6658934925 ) * 69318.64221046682 ),2.803286


In [8]:
# 1) Count how many devices contributed per expression
device_counts = pivot_df.notna().sum(axis=1)

# 2) Compute the max possible entropy = log2(count)
max_entropy = np.log2(device_counts.replace(0, np.nan))

# 3) Scale
entropy_df['entropy_scaled'] = entropy_df['entropy'] / max_entropy

# 4) Fill any NaNs (e.g. when only 0 or 1 device) with 0
entropy_df['entropy_scaled'] = entropy_df['entropy_scaled'].fillna(0)

## Top Discriminating Expressions (by Variance)
Variance is not that important here because we are looking for expressions that are different across devices. However, it can still be useful to see which expressions have the highest variance.

In [9]:
variance_df.head(20)

,variance
expression,
32970.133250076935 * ( e * 21331.302134109705 + sin( pi * 2426.5901943005865 / ( 113385.35342747977 ) - 54949.65054734511 / 0.572849080665037 * -59627.8480884388 + tan( -11797880.755120402 ) / cos( 32991.56475314037 ) ) ) / tan( 91858.34696446847 / cos( 259169.5286292584 - sin( 83803.39611051466 ) - 2282.303712737041 ) ) + pi * 0.3770266817882657,2.244013e+14
144116.27305271316 * pi / e + pi * e * sin( ( tan( 73424.786949128 ) * 0.9439510465599597 / ( 56530.61477244103 ) - pi ) * 255464.21610215 / 7119.375702326182 - 0.5130449610296637 * tan( pi / tan( 55629.497388729535 ) - e * e ) + pi / 80600.1643548241 ) + ( tan( ( 344097.42149705905 ) ) * cos( 66626.91447564634 * 142554.71793764457 + 408804.31088346115 ) * 217339.59344884998 + ( 73348.94143565705 * 19500.567223032776 / 84981.18549244491 + 284612.24463356653 * -25003.225251357995 ) / 827187.116865794 - infinity / e * ( sin( 912573.6511624103 ) + 119707.6811974998 / 0.9168360356707126 ) ) * 130212.05105774822 / ( cos( 2550.9780349103166 / 0.11417809268459678 * tan( 170976.83206274835 ) + 0.3416550566907972 / 129460.99766829397 ) / 46250.55158144567 / 65848.97454139592 ),2.244013e+14
e / 51792.926274419005 / 292754.9351654965 + ( cos( ( 50192.10938532508 ) * 343381.37186629564 / 0.9428970818407834 ) * 0.8273021187633276 / ( 0.12080961861647666 * 0.521072308300063 - 0.29102701018564403 ) ) - tan( sin( 0.2602045340463519 / tan( 66601.06102373435 ) ) * ( 69830.08841963652 * sin( 334295.37086600193 ) ) / pi ) / 117350.57132645075 / ( 0.0020127128809690475 / ( 18032.3337984323 - 0.07703996892087162 / 0.22185992076992989 + 1349727.662409458 * 84720.58775373698 ) ),2.244013e+14
e / 0.8083986975252628 / ( 429444.77165502403 / sin( 98808.98904537983 / cos( 239493.99768523243 ) ) + 0.9500930760987103 / ( 0.19637086032889783 * 0.5169258562382311 * ( 205062.81970092517 ) ) / cos( 37147.44541188763 / ( 841430.1787131258 ) + 0.14586688857525587 * sin( 172499.07805386206 ) ) ) - ( ( cos( 69477.1999171126 ) / e - 0.014860054245218635 / ( 17887.92435889667 ) + 300743.00208643294 * pi * ( 43299.21589925185 ) ) - tan( 1816.1981264709827 / 0.8418901602271944 + 0.5105363873299211 / 175281.54605080388 / 187607.8017576784 ) + infinity / 181885.94153351744 / sin( cos( 131975.91989331678 ) / 56622.784323933505 - ( -54340801.27615317 ) / 104452.04847617635 + 78795.48426028604 * 0.9188502880278975 * ( 24707.355183412452 ) ) ) - 63207.87831575034 * ( 4723.83455082577 / 79856.61634919904 - pi ),2.244013e+14
156780.27200292773 / e * ( 157766.20962136713 + 140960.9830923717 / tan( -25348.602231715395 - ( 547894.3535073622 ) * 870.9039561331771 ) / 0.7385085606947541 ) - ( 40193.629828694735 * 101551.90495629811 * 0.9660667572170496 - 0.9047497091814876 - pi * e ) / ( 0.6578597927000374 - 163248.54100633305 * sin( 199168.54673455376 * tan( 131798.73773057092 ) / cos( 53095.94215184761 ) ) ) / ( pi ),2.244013e+14
( e / sin( ( 7822.046031714993 ) - 247036.81504952986 * 0.383386337896809 ) - 11856.619339385543 * 65417.86335882549 / ( 0.840907184407115 * 222740.33950751647 + e + pi / ( 153634.2365913437 ) ) + ( 113533.46217192124 ) ) * ( ( ( 77509.65101392985 ) - 0.46244007046334445 * ( 138460.2399282894 ) - e * ( 2672.1510906169665 ) * infinity ) / tan( 65845.77103623672 * 5860.264102150098 * 324349.93660747743 ) - 0.4010413638316095 / ( ( 58118.376286106686 ) - e - 0.11916521098464727 / 0.08358802087605 ) ) / pi + tan( 114943.43563296756 ) * 101124.51455956457 * tan( 260577.13418472867 ),2.244013e+14
sin( 120511.62227241456 * 82109.04739416987 + 163933.5672223619 / ( ( 330467.1532101026 ) - 2589147.008702339 / pi ) * 0.20643631252460182 ) / cos( 570863.9401198282 / 12253.879804313407 - 0.4794786931015551 ) / 0.4600610239431262 + cos( 115425.85986848721 / tan( sin( 193521.25273909446 ) + 0.46031457907520235 * 38875.946571342865 * 107861.57008425212 - 73733.34565752839 / sin( 56909.151822983906 ) * 70774.87102534215 ) + 0.39262618822976947 * ( ( 287409.56034314155 ) * 71960.9762548649 * (

## Top Expressions by Entropy
Entropy is a measure of uncertainty or randomness. In this context, it can be used to identify expressions that have a wide range of possible values across devices and are good candidates for fingerprinting.

In [10]:
entropy_df.head(100)

,entropy,entropy_scaled
expression,,
0.836337408516556 / sin( ( 328660.6002077978 * pi + ( 169101.13197941292 ) ) * ( 121897.3210943361 * 59011.5150966451 * 33744.317029065365 + 101130.22947068278 / ( 247623.22206646253 ) / 0.8113027140498161 ) ) + e,2.868240,0.519274
0.26773800421506166 - 47347.7351729909 / 69806.10740005101 - e * tan( pi + 54370.98221018599 / tan( 145.19201052953946 ) * 0.44596617901697755 * 135917.42503542083 ) / pi,2.841861,0.514498
0.09127735486254096 / 735.8453242566477 - sin( 81929.39040861855 / 0.7371127642691135 ) * sin( pi - 0.0050916902255266905 ) * ( pi * 56598.82715263593 - tan( 138618.8207329427 - sin( 87866.92300892183 ) ) * 73963.6481315199 * sin( 38350.44718330214 * 88426.60347159175 * 70748.46378042221 + e * 0.3522350515704602 ) - 155284.45749961227 / 76410.38662212781 / tan( 0.6574289293494076 * cos( 5566943.480322141 ) - e / 27243.048928791104 ) ) + ( e ) * ( 0.767688465770334 - e / sin( 78576.17265717337 * 357461.36174224137 + 81647.98113721207 ) ),2.841861,0.514498
pi * tan( -172362.5349671105 * cos( 106590.18812615762 ) - -135310.96249555165 / 0.43122277199290693 - ( sin( 35.65965735952698 ) + ( 39092.58851640222 ) / 143281.34438369097 ) * 25952.2169060161 * ( pi ) ),2.822718,0.511032
0.7887798345182091 * sin( 82189.05545839571 / 0.18517374992370605 * 0.008089403854683042 ) / tan( tan( 109291.67664288744 * ( 36295.7350248317 ) - 204664.3786005878 - 209845.36376121573 * 0.7465927922166884 ) * 184123.336033595 ) + ( sin( ( 131293.05520239723 ) - 150286.96096293302 * 0.8082891888916492 ) / 0.777242538286373 + 0.5813473116140813 ) * 0.6146595270838588 * tan( 0.7512196360621601 / 8514.927909941202 + ( 136784.9486422678 ) / ( 0.5166313732042909 * 105161.13063391473 / 0.6193476456683129 ) + 0.599509299499914 * 467214.32023847103 ),2.813943,0.509444
...,...,...
30549.369323299852 - tan( cos( ( 133937.82263119414 ) + ( 42804.329956443034 ) * 198155.63365702258 ) / ( pi / 180891.29996967732 * e ) ) / cos( ( ( 175666.9995266514 ) / 30432.56774929175 - 2330379.3333656355 * 120531.22164096583 + 19929.379743674883 ) / e / ( pi + infinity - 0.3969584743026644 / 66915.3316584154 ) - tan( 0.8694365243427455 ) * 0.44176260964013636 ) * ( 75021.91658686954 / 0.074489502934739 - 345415.09424076934 / -77554.52237619222 * -806159.8675273553 - 146054.47829046502 * -infinity ) * ( tan( 116079.21773615229 ) * 22349.049061626276 ) - e / 0.18367218971252441 / 51623.177369509576 + 0.1396392728202045,2.444803,0.442613
cos( tan( 0.26706047309562564 / 0.45351544907316566 - e * e * 31473.04196084083 ) - ( 0.6271405860316008 - ( 132822.9796098632 ) * 1919222.097897004 ) / 0.7984978766180575 + 100681.73029021188 / e ) / tan( 201573.92452105874 - ( ( 128458.4391245637 ) / sin( 319305.6848595787 ) ) ) + ( sin( tan( 87742.31228712824 ) * 245158.79310323764 - 243944.1171910585 / ( 257528.7905082424 ) ) / 47780.21444322079 / 0.21108739636838436 - ( e / 0.45928976545110345 * 222805.040360132 + 105039.02733940909 / 0.1630849363282323 ) * 65579.63034176256 ) * cos( ( 47411.37604300482 * 123222.25146908814 ) / 0.5969401963520795 / ( -79737.07178237406 ) - 58298.025019870794 / -144592.6147087766 ),2.444803,0.442613
307638.73848843406 / ( 100025.69481942254 / pi - ( sin( 116188.06380455998 ) + 144898.20216744955 * 263298.55150673416 * 40629.18837815485 ) * 2880737.9293186096 ) * -infinity,2.444803,0.442613


In [16]:
# find values for the top 10 expressions
top_10_entropy = entropy_df.head(10).index
top_10_entropy_df = pivot_df.loc[top_10_entropy]
top_10_entropy_df
# top_10_entropy_df = top_10_entropy_df.transpose()
# top_10_entropy_df.columns = top_10_entropy
# top_10_entropy_df = top_10_entropy_df.reset_index()
# top_10_entropy_df.columns = ['expression'] + list(top_10_entropy)
# top_10_entropy_df = top_10_entropy_df.set_index('expression')
# top_10_entropy_df = top_10_entropy_df.transpose()
# top_10_entropy_df = top_10_entropy_df.fillna('')
# top_10_entropy_df

device,iphone13_ios_brave_1.61.1,iphone13_ios_chrome_121,iphone13_ios_edge_121,iphone13_ios_ghostery_3.2,iphone13_ios_opera_4.5.0,iphone13_ios_safari_17.2.1,iphone13_windows_firefox_122,macbook_macos_brave_v1.62.156,macbook_macos_chrome_121,macbook_macos_edge_121,...,win10vm_windows_ghostery_2023.10,win10vm_windows_opera_106,win10vm_windows_tor_13.0.9,win11vm_windows_brave_v1.62.156,win11vm_windows_chrome_121,win11vm_windows_edge_121,win11vm_windows_firefox_122,win11vm_windows_ghostery_2023.10,win11vm_windows_opera_106,win11vm_windows_tor_13.0.9
expression,,,,,,,,,,,,,,,,,,,,,
0.836337408516556 / sin( ( 328660.6002077978 * pi + ( 169101.13197941292 ) ) * ( 121897.3210943361 * 59011.5150966451 * 33744.317029065365 + 101130.22947068278 / ( 247623.22206646253 ) / 0.8113027140498161 ) ) + e,1.765625,1.765625,1.765625,1.765625,1.765625,1.765625,1.765625,1.677720e+07,1.677720e+07,1.677720e+07,...,0.000000,1.765620,1.86667,3.355440e+07,3.355440e+07,3.355440e+07,0.000000,0.000000,1.765620,1.86667
0.26773800421506166 - 47347.7351729909 / 69806.10740005101 - e * tan( pi + 54370.98221018599 / tan( 145.19201052953946 ) * 0.44596617901697755 * 135917.42503542083 ) / pi,7.937500,7.937500,7.937500,7.937500,7.937500,7.937500,7.937500,7.953120e+00,7.953120e+00,7.953120e+00,...,0.050000,7.921880,0.00000,7.953120e+00,7.953120e+00,7.953120e+00,0.050000,0.050000,7.921880,0.00000
0.09127735486254096 / 735.8453242566477 - sin( 81929.39040861855 / 0.7371127642691135 ) * sin( pi - 0.0050916902255266905 ) * ( pi * 56598.82715263593 - tan( 138618.8207329427 - sin( 87866.92300892183 ) ) * 73963.6481315199 * sin( 38350.44718330214 * 88426.60347159175 * 70748.46378042221 + e * 0.3522350515704602 ) - 155284.45749961227 / 76410.38662212781 / tan( 0.6574289293494076 * cos( 5566943.480322141 ) - e / 27243.048928791104 ) ) + ( e ) * ( 0.767688465770334 - e / sin( 78576.17265717337 * 357461.36174224137 + 81647.98113721207 ) ),279.093750,279.093750,279.093750,279.093750,279.093750,279.093750,279.093750,2.749220e+02,2.749220e+02,2.749220e+02,...,93.016700,279.094000,94.48330,2.749220e+02,2.749220e+02,2.749220e+02,93.016700,93.016700,279.094000,94.48330
pi * tan( -172362.5349671105 * cos( 106590.18812615762 ) - -135310.96249555165 / 0.43122277199290693 - ( sin( 35.65965735952698 ) + ( 39092.58851640222 ) / 143281.34438369097 ) * 25952.2169060161 * ( pi ) ),6.484375,6.484375,6.484375,6.484375,6.484375,6.484375,6.484375,6.492190e+00,6.492190e+00,6.492190e+00,...,2.983330,6.484380,3.18333,6.484380e+00,6.484380e+00,6.484380e+00,2.983330,2.983330,6.484380,3.18333
0.7887798345182091 * sin( 82189.05545839571 / 0.18517374992370605 * 0.008089403854683042 ) / tan( tan( 109291.67664288744 * ( 36295.7350248317 ) - 204664.3786005878 - 209845.36376121573 * 0.7465927922166884 ) * 184123.336033595 ) + ( sin( ( 131293.05520239723 ) - 150286.96096293302 * 0.8082891888916492 ) / 0.777242538286373 + 0.5813473116140813 ) * 0.6146595270838588 * tan( 0.7512196360621601 / 8514.927909941202 + ( 136784.9486422678 ) / ( 0.5166313732042909 * 105161.13063391473 / 0.6193476456683129 ) + 0.599509299499914 * 467214.32023847103 ),0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,3.671880e-01,3.671880e-01,3.671880e-01,...,0.000000,0.593750,0.00000,3.593750e-01,3.593750e-01,3.593750e-01,0.000000,0.000000,0.593750,0.00000
( 819089.5583285954 / 56608.20596055245 ) / cos( pi + sin( 76236.68535730825 / 40736.93352735403 ) ) / 130187.78693887388 * 649103.1720475996 / sin( ( 211870.47864979794 ) * 493475.8066673059 ),938.671875,938.671875,938.671875,938.671875,938.671875,938.671875,938.671875,9.386170e+02,9.386170e+02,9.386170e+02,...,2107.980000,938.672000,2107.98000,9.386090e+02,9.386090e+02,9.386090e+02,2107.980000,2107.980000,938.672000,2107.98000
2177121.019011976 / 134882.69539745056 / 0.9244347715284675 - ( cos( e * ( 191596.40883588613 ) + 262619.3231425485 * 274051.0595224638 ) / 0.328251866158098 ) * sin( ( 11588.896119517485 * 32314.82891819235 * cos( 20434908.45764997 ) ) * pi * 1

In [22]:
#for each expression, what are the unique values?
for expr in top_10_entropy:
    print(f"Expression: {expr}")
    print(rec_df[rec_df['expression'] == expr].drop(columns=['expression', 'device']).drop_duplicates())
    print()

Expression: 0.836337408516556 /  sin(  ( 328660.6002077978 *  pi  +  ( 169101.13197941292 )  )  *  ( 121897.3210943361 * 59011.5150966451 * 33744.317029065365 + 101130.22947068278 /  ( 247623.22206646253 )  / 0.8113027140498161 )  )  +  e 
              result
3141    1.765625e+00
73141   1.677720e+07
103141  0.000000e+00
123141  1.765620e+00
153141  1.278260e+07
173141  1.767860e+00
223141  2.064890e+07
233141  1.766600e+00
256141  3.355440e+07
386141  1.866670e+00

Expression: 0.26773800421506166 - 47347.7351729909 / 69806.10740005101 -  e  *  tan(  pi  + 54370.98221018599 /  tan( 145.19201052953946 )  * 0.44596617901697755 * 135917.42503542083 )  /  pi 
         result
2081    7.93750
72081   7.95312
102081  0.05000
122081  7.92969
142081  0.00000
152081  7.95238
172081  7.92857
222081  7.95192
232081  7.92773
305081  7.92188

Expression: 0.09127735486254096 / 735.8453242566477 -  sin( 81929.39040861855 / 0.7371127642691135 )  *  sin(  pi  - 0.0050916902255266905 )  *  (  pi  * 5659